In [60]:
# importing libraries

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

import torch 
from torch import nn 

In [61]:
# Define the batch size and model save paths

BATCH_SIZE = 128
PATH = 'Models/01_Direct_10M/'

In [62]:
# Loading the dataset

data = pd.read_csv('ik_dataset_1.csv')
data.head()

,joint_0,joint_1,joint_2,joint_3,joint_4,joint_5,x,y,z
0,2.675345,0.691554,2.023822,0.245126,1.865613,0.730238,-0.318110,0.186086,0.750141
1,-1.787474,0.297828,-0.546427,-0.220475,0.097833,0.294982,-0.130555,-0.603010,-0.347686
2,2.572510,-2.728705,1.440937,3.092787,-0.589794,-2.233969,0.350076,-0.227156,-0.400677
3,-0.323000,-2.374566,-3.110326,-2.896323,2.700566,-1.937711,0.350737,-0.106469,-0.210679
4,-0.931573,-1.593713,2.104227,-1.325443,1.427286,-0.029970,0.459477,-0.457196,-0.451925


In [63]:
# Normalising the data 

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data))
data.columns = ['joint_0','joint_1','joint_2','joint_3','joint_4','joint_5','x','y','z']
data.head()

,joint_0,joint_1,joint_2,joint_3,joint_4,joint_5,x,y,z
0,0.925907,0.610119,0.822168,0.539028,0.796835,0.616467,0.347672,0.584083,0.886867
1,0.215465,0.547441,0.413060,0.464921,0.515361,0.547166,0.435933,0.209645,0.236153
2,0.909537,0.065643,0.729390,0.992273,0.405874,0.144506,0.662112,0.387993,0.204744
3,0.448596,0.122019,0.004962,0.039022,0.929781,0.191677,0.662424,0.445261,0.317361
4,0.351717,0.246324,0.834966,0.289049,0.727043,0.495427,0.713595,0.278836,0.174368


In [64]:
# Device agnostic code

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device

'cuda'

In [65]:
# Converting data to tensors 

xtest = torch.Tensor(data[['x','y','z']].to_numpy())
ytest = torch.Tensor(data[['joint_0','joint_1','joint_2','joint_3','joint_4','joint_5']].to_numpy())

xtest.shape, ytest.shape

(torch.Size([10000, 3]), torch.Size([10000, 6]))

In [66]:

# Defining the model 

class InverseKinematicsABBIRB140(nn.Module):
    
    """ This is a model structure specifically designed to calculate the inverse kinematics of the industrial robot ABB IRB 140, but can be used for any robot
        This has a layer structure of input -> 64 -> 128 -> 64 -> output with non-linear activation and normalisation layers
    Args:
        input_features (int): defines the number of features given as input to the model
        output_features (int): defines the number of features the model outputs
    """
    
    def __init__(self, input_features, output_features):
        super().__init__()
        self.layer_stack_1 = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_2 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_3 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_4 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout(),
            nn.Linear(in_features=128, out_features=output_features)
        )
    
    def forward(self, x):
        x = self.layer_stack_1(x)
        x = self.layer_stack_2(x)
        x = self.layer_stack_3(x)
        x = self.layer_stack_4(x)
        return x   
    

In [67]:
# Instantiating various models used 

model = InverseKinematicsABBIRB140(input_features=3,output_features=6)
model.load_state_dict(torch.load(f=PATH+'model_0.pth'))

<All keys matched successfully>

In [68]:
# Calculate all the joint angles from our models

model.eval()
with torch.inference_mode():
    ypred = model(xtest)
    

In [69]:
ypred[:5], ytest[:5]

(tensor([[0.7606, 0.1590, 0.3960, 0.5019, 0.5044, 0.5104],
         [0.2310, 0.7022, 0.4747, 0.5008, 0.5026, 0.4905],
         [0.5942, 0.7389, 0.7678, 0.5015, 0.5122, 0.5012],
         [0.6140, 0.6540, 0.8947, 0.4963, 0.5168, 0.5016],
         [0.3840, 0.7324, 0.4675, 0.5022, 0.5037, 0.4931]]),
 tensor([[0.9259, 0.6101, 0.8222, 0.5390, 0.7968, 0.6165],
         [0.2155, 0.5474, 0.4131, 0.4649, 0.5154, 0.5472],
         [0.9095, 0.0656, 0.7294, 0.9923, 0.4059, 0.1445],
         [0.4486, 0.1220, 0.0050, 0.0390, 0.9298, 0.1917],
         [0.3517, 0.2463, 0.8350, 0.2890, 0.7270, 0.4954]]))

In [70]:
loss_fn_1 = nn.MSELoss()

In [71]:
loss_fn_1(ypred, ytest)

tensor(0.1127)

In [72]:
loss_fn_2 = nn.L1Loss()

In [73]:
loss_fn_2(ypred, ytest)

tensor(0.2747)

### Final Loss

1) Direct Approach :
    1) MSE Loss : 0.1127
    2) L1 Loss :0.2747
2) Sequential Approach :
    1) MSE Loss : 0.0620
    2) L1 Loss : 0.1975